<a href="https://colab.research.google.com/github/herr-hollis/Analyse-Telco-Custer-Churn-using-XGBoost/blob/main/Data_Workshop_Covid_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the Data Visualization Workshop!



## Import packages

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Import Malaysia's Covid cases, vaccination by state and country

These data sources are from the goverment and are updated on a daily basis. Some dataset might only have t-1 or t-2 data.

In [ ]:
# Malaysia Population Dataset
population_df = pd.read_csv('https://raw.githubusercontent.com/CITF-Malaysia/citf-public/main/static/population.csv')

# Malaysia Covid Cases Dataset
case_my = 'https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/cases_malaysia.csv'
case_my = pd.read_csv(case_my, parse_dates=['date'])

# Malaysia Covid Death Cases Dataset
death_my = 'https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/deaths_malaysia.csv'
death_my = pd.read_csv(death_my, parse_dates=['date'])

# Malaysia Covid Vaccination Dataset
vax_my = 'https://raw.githubusercontent.com/CITF-Malaysia/citf-public/main/vaccination/vax_malaysia.csv'
vax_my = pd.read_csv(vax_my, parse_dates=['date'])

# Malaysia Covid Vaccination by State Dataset
vax_state = 'https://raw.githubusercontent.com/CITF-Malaysia/citf-public/main/vaccination/vax_state.csv'
vax_state = pd.read_csv(vax_state)

In [ ]:
# Show top two rows
population_df.head(2)

,state,idxs,pop,pop_18,pop_60
0,Malaysia,0,32657400,23409600,3502000
1,Johor,1,3781000,2711900,428700


## Check Malaysia's latest covid cases

In [ ]:
# Show top two rows
case_my.tail(2)

,date,cases_new,cases_import,cases_recovered,cluster_import,cluster_religious,cluster_community,cluster_highRisk,cluster_education,cluster_detentionCentre,cluster_workplace
600,2021-09-16,18815,7,16939,0.0,4.0,456.0,47.0,45.0,54.0,775.0
601,2021-09-17,17577,3,22970,0.0,3.0,432.0,31.0,13.0,20.0,1055.0


## Check Malaysia's latest death data

In [ ]:
# Show top two rows
death_my.tail(2)

,date,deaths_new,deaths_new_dod,deaths_bid,deaths_bid_dod
548,2021-09-16,346,41,105,0
549,2021-09-17,388,9,64,0


## Check Malaysia vaccination data

In [ ]:
# Show top two rows
vax_my.tail(2)

,date,daily_partial,daily_full,daily,cumul_partial,cumul_full,cumul,cumul_partial_child,cumul_full_child,pfizer1,pfizer2,sinovac1,sinovac2,astra1,astra2,cansino,pending
204,2021-09-16,62108,140190,202298,21783871,17973545,39685414,86954,0,39815,80274,7178,33376,386,23043,2210,16016
205,2021-09-17,75986,162016,238002,21861904,18135561,39923416,116322,0,68123,101285,6950,34810,606,22376,2047,1805


In [ ]:
# Show top two rows
vax_state.tail(2)

,date,state,daily_partial,daily_full,daily,cumul_partial,cumul_full,cumul,cumul_partial_child,cumul_full_child,pfizer1,pfizer2,sinovac1,sinovac2,astra1,astra2,cansino,pending
3294,2021-09-17,W.P. Labuan,45,75,120,73837,66984,140821,2461,0,43,70,2,5,0,0,0,0
3295,2021-09-17,W.P. Putrajaya,139,685,824,128688,123078,251766,0,0,139,658,0,22,0,0,0,5


## Merging/joining our vaccination data with cases and death data
So we can have combined or holistic view of vaccination, covid and death cases as both of the dataset shares the same date.

In [ ]:
final_data = vax_my.merge(case_my, on='date', how='left')

In [ ]:
final_data = final_data.merge(death_my, on='date', how='left')

In [ ]:
final_data["malaysia_pop"] = population_df.iloc[0,2]

In [ ]:
# Show top two rows
final_data.head(2)

,date,daily_partial,daily_full,daily,cumul_partial,cumul_full,cumul,cumul_partial_child,cumul_full_child,pfizer1,pfizer2,sinovac1,sinovac2,astra1,astra2,cansino,pending,cases_new,cases_import,cases_recovered,cluster_import,cluster_religious,cluster_community,cluster_highRisk,cluster_education,cluster_detentionCentre,cluster_workplace,deaths_new,deaths_new_dod,deaths_bid,deaths_bid_dod,malaysia_pop
0,2021-02-24,60,0,60,60,0,60,0,0,60,0,0,0,0,0,0,0,3545,1,3331,0.0,0.0,70.0,19.0,0.0,29.0,2338.0,12,5,1,0,32657400
1,2021-02-25,1146,0,1146,1206,0,1206,0,0,1146,0,0,0,0,0,0,0,1924,6,3752,0.0,1.0,76.0,26.0,0.0,105.0,442.0,13,6,2,1,32657400


## Checking Null Values

In [ ]:
final_data.isnull().sum()

date                       0
daily_partial              0
daily_full                 0
daily                      0
cumul_partial              0
cumul_full                 0
cumul                      0
cumul_partial_child        0
cumul_full_child           0
pfizer1                    0
pfizer2                    0
sinovac1                   0
sinovac2                   0
astra1                     0
astra2                     0
cansino                    0
pending                    0
cases_new                  0
cases_import               0
cases_recovered            0
cluster_import             0
cluster_religious          0
cluster_community          0
cluster_highRisk           0
cluster_education          0
cluster_detentionCentre    0
cluster_workplace          0
deaths_new                 0
deaths_new_dod             0
deaths_bid                 0
deaths_bid_dod             0
malaysia_pop               0
dtype: int64

## Changing date display format (Necessary for Google Sheet)

In [ ]:
final_data['date'] = final_data.date.apply(lambda x: x.strftime('%Y%m%d')).astype(int)

In [ ]:
#Show top two
final_data.head(2)

,date,daily_partial,daily_full,daily,cumul_partial,cumul_full,cumul,cumul_partial_child,cumul_full_child,pfizer1,pfizer2,sinovac1,sinovac2,astra1,astra2,cansino,pending,cases_new,cases_import,cases_recovered,cluster_import,cluster_religious,cluster_community,cluster_highRisk,cluster_education,cluster_detentionCentre,cluster_workplace,deaths_new,deaths_new_dod,deaths_bid,deaths_bid_dod,malaysia_pop
0,20210224,60,0,60,60,0,60,0,0,60,0,0,0,0,0,0,0,3545,1,3331,0.0,0.0,70.0,19.0,0.0,29.0,2338.0,12,5,1,0,32657400
1,20210225,1146,0,1146,1206,0,1206,0,0,1146,0,0,0,0,0,0,0,1924,6,3752,0.0,1.0,76.0,26.0,0.0,105.0,442.0,13,6,2,1,32657400


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
import gspread_dataframe as gd

# Connecting with `gspread` here

gc = gspread.authorize(GoogleCredentials.get_application_default())

## Create Google Sheet called Covid.
Let's go to the file once it has been created. We will manually create two working sheet to understand how to separate our data better.
Create:
1. Data sheet (Vaccination + Cases)
2. State (Vaccination by state)

In [ ]:
sh = gc.create('Covid')

## Saving our vaccination and cases data into worksheet called Data

In [ ]:
ws = gc.open("Covid").worksheet("Data")
existing = gd.get_as_dataframe(ws)

gd.set_with_dataframe(ws, final_data)

## Saving our vaccination by state data into worksheet called State

In [ ]:
ws = gc.open("Covid").worksheet("State")
existing = gd.get_as_dataframe(ws)
gd.set_with_dataframe(ws, vax_state)

# Let's head over to Data Studio to start visualizing it :D